In [2]:
import os 
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Note:
# torch.nn中包含了所有用于构建神经网络的module。并且Pytorch中的每个module都继承于nn.Module.

In [5]:
'''Get device for training'''
# 1.from the offical website, which can not work
# device = torch.accelerator.current_accelerator().type() if torch.accelerator.is_available() else "cpu"
# print(device)

# 2.From gpt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
'''Define the Class'''
class NeuralNetwork(nn.Module): #自定义的神经网络必须要subclass nn.Module
    def __init__(self):
        # 在__init__函数中初始化神经网络的layers
        super().__init__() #千万不要忘记调用父类的__init__()函数，注册所有的参数，并且训练过程中才能进行梯度的计算和更新
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        # 在forward函数中就是将输入走一遍我们在__init__()中定义的网络结构，得到最后的输出
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# 注意：
# 1.不要直接call model.forward()!

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(2, 28, 28, device = device)
logits = model(X) 
#模型返回的是一个2维的tensor，第一维对应着batch_size维（也就是输入样本的个数），
# 第二维对应着对于10个类别分别的输出结果
print(logits.shape)
print(logits)

pred_probab = nn.Softmax(dim = 1)(logits) #在指定的维度上进行softmax操作
print(pred_probab)
print(pred_probab.shape)

y_pred = pred_probab.argmax(1) #在指定的维度上，选取最大值对应的下标
print(y_pred)
print(y_pred.shape)

torch.Size([2, 10])
tensor([[-0.0190, -0.0934, -0.0719, -0.0672,  0.1167,  0.1339,  0.0328,  0.0424,
         -0.0047, -0.0238],
        [ 0.0073, -0.0593, -0.0847, -0.0442,  0.1238,  0.0742,  0.0466,  0.0382,
          0.0070, -0.0175]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0974, 0.0904, 0.0924, 0.0928, 0.1116, 0.1135, 0.1026, 0.1036, 0.0988,
         0.0969],
        [0.0996, 0.0932, 0.0909, 0.0946, 0.1119, 0.1065, 0.1036, 0.1028, 0.0996,
         0.0972]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
torch.Size([2, 10])
tensor([5, 4], device='cuda:0')
torch.Size([2])


In [18]:
'''Model Layers'''
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [20]:
# 1. nn.Flatten
# nn.Flaten()的作用是在保持batch_size维度不变的情况下，将剩余维度从 2D 的height * width
# 图像 展平为一维的包含height * width个值的连续数组。
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.shape)

torch.Size([3, 784])


# 1. nn.Flatten()的介绍
## 1.1 作用
&nbsp;&nbsp;&nbsp;&nbsp;将多维的tensor准换位1维的向量，同时保持batch的维度不变。<br>
&nbsp;&nbsp;&nbsp;&nbsp;通常在模型的卷积层之后，全连接层fully connected(dense) layers之前使用。因为CNN通过卷积来处理图像，最后会产生一个高纬的特征图，为了将这些特征送入全连接层(nn.Linear)，我们必须将特征图展平。
## 1.2 语法
```python
    nn.Flatten(start_dim, end_dim) #可以控制需要展平的维度范围，如果不显示指定，那么默认从start_dim = 1（因为要跳过dim = 0的batch维）到最后一维。
```
## 1.3 为什么需要Flatten
&nbsp;&nbsp;&nbsp;&nbsp;主要的原因是nn.Linear(in_features, out_features)只接受2维(batch, features)的输入，而CNN卷积层的输出通常是(batch, channels, height, width)的高纬向量，所以必须要使用nn.Flatten()或者 CNN输出.view()/.reshape() 来转换tensor的形状。

## 1.4 view()和reshape()函数的区别与联系
### 1.4.1 view()
&nbsp;&nbsp;&nbsp;&nbsp;（1）定义： .view()会返回一个指定形状的新tensor，但是仍然和原来的tensor共享内存（改变两者中的其一的话，另一个跟着受影响），它并不会创建一个新的副本。<br>
&nbsp;&nbsp;&nbsp;&nbsp;（2）特点： .view()要求tensor必须在内存空间中是连续存储的，否则需要先调用.contiguous()函数之后，才能调用.view()；.contiguous()的作用是在内存的连续空间中创建一个新的副本，使得新建的副本是连续存储的。使用.is_contiguous()来判断是否连续存储。<br>
它不会在内存中分配新的空间，而仅仅是将原始数据修改到指定的shape中。
### 1.4.2 reshape()
&nbsp;&nbsp;&nbsp;&nbsp;（1）定义：.reshape()会返回一个指定形状的新tensor，并且可以自动解决原始tensor的内存空间不连续的问题，因为它会自动创建一个新的备份之后再进行reshape操作，可以理解为是contiguous()和view()的结合。

### 1.4.3 两者之间的联系
&nbsp;&nbsp;&nbsp;&nbsp;它们都是用来reshape一个tensor到指定形状的函数。如果都是用于和nn.Flatten()一样的展平功能，务必使用 CNN输出tensor.view(CNN输出tensor.shape[0], -1) 或者 CNN输出tensor.reshape(CNN输出tensor.shape[0], -1)。 因为两个函数都会默认保持batch维度不变，但是不知道batch维度有多少个sample，需要显示指定。 -1是表明pytorch会自动计算reshape之后的第二个维度应该是多大，它通常是除了batch维度的其他维度的数组的乘积。

### 1.4.4 pytorch中的连续存储是什么意思？
![](./images/pytorch中的连续存储.jpg) <br>
![](./images/pytorch中的连续存储_1.jpg)

In [ ]:
# 2.nn.Linear()
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.shape)

torch.Size([3, 20])


# 1.nn.Linear()的详细介绍
nn.Linear()代表了全连接层或者也叫Dense层，主要用于神经网络中的线性变化。
## 1.1 理论
![](./images/Linear层的理论知识.jpg)

## 1.2 语法
```python
nn.Linear(in_features = xx, out_features = xx, bias = True)
#Note：
# 一般在Batch Normalization（因为已经有bias了）和数据是中心分布的时候，需要把bias设置为False。
#并且nn.Linear()只会对输入的tensor，形状为(batch, feature)中的feature进行操作，而会保持batch不变

#nn.Linear()的等价实现——torch.matmul()：
y = torch.matmul(x, W.T) + b 
#但是这个实现方式无法实现梯度更新
```
## 1.3 作用
（1）将高维特征压缩为低维特征：比如self.fc1 = nn.Linear(4, 3)就把输入的4个特征转换为了3个特征。<br>
（2）从特征中学习abstract patterns（抽象模式）：通过trainable的权重更新来学习<br>
（3）将特征映射到输出，比如网络的最后一个Linear层可以把特征映射到最终的分类结果或者回归结果。<br>

## 1.4 实际的应用场景
![](./images/Linear层的应用场景-CNN的分类头.jpg)
![](./images/Linear层的应用场景-分类头.jpg)
![](./images/Linear层的应用场景-回归.jpg)


In [ ]:
# 3.nn.ReLU
# 非线性的激活通常用于在模型的输入和输出之间简历复杂的非线性映射关系。它们通常在线性变换之后
# 应用，以引入nonlinearity(非线性)，帮助神经网络学习更加复杂多样的现象和特征。
print(hidden1)
hidden1 = nn.ReLU()(hidden1)
print(hidden1)

tensor([[ 2.4768e-01, -3.4506e-01, -3.9492e-01,  1.1021e-01,  3.8575e-01,
         -1.7673e-01, -5.2702e-02,  3.9015e-01,  1.5172e-01, -3.2451e-01,
         -3.3576e-01,  1.1361e-01,  4.9022e-01,  1.0703e-02, -3.0100e-01,
         -2.1259e-01, -5.5687e-02, -2.7264e-02,  1.5141e-01,  1.9673e-02],
        [-1.6089e-01, -2.9776e-01, -2.2384e-01, -3.0235e-02,  2.8877e-01,
         -2.9372e-01, -1.8886e-01,  6.7092e-01, -1.6420e-01, -2.9050e-01,
          6.7656e-02,  1.5294e-01,  7.4763e-01, -2.7335e-02, -4.8561e-01,
          1.6684e-01, -2.7199e-01,  1.6221e-01,  2.0042e-01, -2.6755e-01],
        [ 1.1585e-01,  5.5218e-04,  4.5810e-02, -5.0402e-01,  2.3352e-02,
         -1.2399e-01, -6.7554e-02,  1.7813e-01,  2.0059e-01,  3.6634e-02,
         -2.2618e-01,  1.1012e-01,  3.6806e-01, -1.4018e-01, -1.8711e-01,
         -8.3868e-02,  9.0989e-03,  3.6996e-02, -2.9644e-01,  7.0817e-02]],
       grad_fn=<AddmmBackward0>)
tensor([[2.4768e-01, 0.0000e+00, 0.0000e+00, 1.1021e-01, 3.8575e-01, 0.0000

# 1 nn.ReLU()的介绍
## 1.1 理论
![](./images/ReLU的理论.jpg)

## 1.2 特点
### 1.2.1 优点
（1）防止梯度消失，与sigmoid和tanh不同，ReLU不会 <br>
（2）计算友好，无需指数计算<br>
（3）支持稀疏表示：用0表示负数，可以实现稀疏激活<br>
（4）加速训练速度<br>
### 1.2.2 缺点以及对应的解决办法
（1）ReLU失活问题：因为如果过多的神经元输出为0，那么它们就会变得inactive。解决办法是使用LeakyReLU, PReLU, or ELU。<br>
（2）对于超大输入的输出爆炸问题：因为ReLU的正数部分的输出是没有上限的。解决办法是使用BatchNorm或者Gradient Clipping.<br>
（3）非中心分布问题：因为ReLU的输出永远是非负数，所以有的时候tanh或者sigmoid会更好。<br>
### 1.2.3 其他可替换的激活函数
![](./images/ReLU的替换函数.jpg)

In [ ]:
# 4.nn.Softmax()
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

# 1.Softmax()介绍
## 1.1 概念和理论
nn.Softmax()函数就是将tensor（通常是模型输出的logits）转换为一个概率分布。具体来讲，就是将tensor中的每个数值转换为一个0到1之间的概率，并且tensor中这些概率的和为1。通常用于多分类任务中。<br>
![](./images/Softmax函数的理论.jpg)

## 1.2 语法
```python
torch.nn.Softmax(dim = None) #dim指定的是要在哪个维度上应用Softmax函数
#接受的输入的shape必须是(batch, num_classes)，输出的shape和输入的shape是一样的，只不过所有的数值都被转换为了概率，并且概率的和为1.
```

# 1.torch.nn中常用的激活函数
![](./images/常用的激活函数总表.jpg)
## 1.1 Sigmoid()
![](./images/Sigmoid激活函数.jpg)

## 1.2 Tanh()
![](./images/Tanh激活函数.jpg)

## 1.3 GELU()
![](./images/GELU激活函数.jpg)

# 2.梯度消失问题
## 2.1 梯度的计算过程
![](./images/梯度计算的过程.jpg)
 
## 2.2 梯度消失的定义
梯度消失（Vanishing Gradient）指的是在反向传播过程中用于更新权重的梯度（也就是导数derivatives），随着在神经网络的层中不断传播，变得非常小，甚至接近于0的一种现象。最终会导致模型的参数停止更新并且训练的速度会比变慢甚至停止。这种现象在具有大量隐藏层的DNN中非常常见。

## 2.3 梯度消失的成因
主要的原因是每一层的梯度的计算，是由后一层的梯度乘上当前层的激活函数的导数得到的。如果当前层的激活函数的导数的值非常小，那么就会导致想乘之后的结果非常小，假设小于1，那么迭代进行下去，从模型的后面层往前层传递的过程中，就会造成梯度越来越小，最后接近0，导致模型中靠近输入的前几层的梯度很小，更新非常缓慢，导致模型也很难有效地学习。<br>
常见的导致梯度消失的例子：<br>
主要是由于Sigmoid()和Tanh()激活函数。<br>
![](./images/梯度消失的常见案例.jpg)

## 2.4 解决梯度消失的办法
![](./images/解决梯度消失的办法.jpg)
![](./images/解决梯度消失的办法_1.jpg)




In [25]:
# 5.nn.Sequential
seq_modules = nn.Sequential(
    flatten, 
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(seq_modules)
print(logits)
print(logits.shape)

# Note:
# 1.nn.Sequential()就是一个模块的有序容器，数据会按照在容器中定义的模块顺序流动。
# 2.使用了nn.Sequential()之后，就不用定义forward（）函数了。

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=20, bias=True)
  (2): ReLU()
  (3): Linear(in_features=20, out_features=10, bias=True)
)
tensor([[-0.0164, -0.0524, -0.0093, -0.2163, -0.0078, -0.1642, -0.1312,  0.3871,
          0.1448,  0.0935],
        [-0.1388, -0.0547, -0.0081, -0.2281, -0.0429, -0.0980, -0.1004,  0.3253,
         -0.0158,  0.1611],
        [ 0.0126, -0.0093, -0.0522, -0.2624, -0.0634, -0.1017, -0.0645,  0.4176,
          0.0251,  0.0177]], grad_fn=<AddmmBackward0>)
torch.Size([3, 10])


In [ ]:
'''Model Parameters'''
# 神经网络中参数化之后的layers，都会与weights和bias联系起来，并且会在训练过程中被更新。
# 模型继承nn.Module会自动地跟踪所有定义在模型中的fiels，并且使得所有的参数都变得可获取，通过model.parameters()
# 或者model.named_parameters()函数。
print(model)

for name, param in model.named_parameters():
    print(f"{name} | {param.size()} | {param[:2]}")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
linear_relu_stack.0.weight | torch.Size([512, 784]) | tensor([[-0.0042,  0.0113, -0.0198,  ...,  0.0066, -0.0220, -0.0323],
        [-0.0302,  0.0248,  0.0122,  ...,  0.0182,  0.0243,  0.0186]],
       device='cuda:0', grad_fn=<SliceBackward0>)
linear_relu_stack.0.bias | torch.Size([512]) | tensor([0.0326, 0.0207], device='cuda:0', grad_fn=<SliceBackward0>)
linear_relu_stack.2.weight | torch.Size([512, 512]) | tensor([[-0.0196, -0.0412, -0.0261,  ...,  0.0192,  0.0398,  0.0390],
        [-0.0312, -0.0328,  0.0093,  ...,  0.0157,  0.0328, -0.0114]],
       device='cuda:0', grad_fn=<SliceBackward0>)
linear_relu_stack.2.bias | torch.Size([512]) | tensor([ 0.0209, -0.0061], de

# 1.model.parameters()和model.named_parameters()的区别和联系

## 1.1 联系
两者都是用来获取模型的可训练(Trainable)参数的.

## 1.2 差别
（1）model.parameters()只返回parameter tensors的迭代器。而name_parameters()会返回(name, parameter)的迭代器。

## 1.3 各自的使用情况
（1）parameters（）通常和optimizer配合使用。optimizer = torch.optim.Adam(model.parameters())
（2）name_parameters()通常用于检查模型的各个层的参数，或者是用于加载或者保存指定层的时候（比如要冻结某些层的参数）：
```python 
    for name, param in model.named_parameters():
        if "0" in name:  # Freeze the first Linear layer
            param.requires_grad = False
```